In [ ]:
import findspark
findspark.init('/home/ubuntu/spark-3.2.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('BDAS').getOrCreate()
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pyspark.pandas as ps

In [ ]:
df = spark.read.csv('textdata.csv', header=True, inferSchema=True)

In [ ]:
df.printSchema()

In [ ]:
df = df.drop('id', 'Age')

In [ ]:
df = df.withColumn('Inflight wifi service',df['Inflight wifi service'].cast('string'))
df = df.withColumn('Ease of Online booking',df['Ease of Online booking'].cast('string'))
df = df.withColumn('Gate location',df['Gate location'].cast('string'))
df = df.withColumn('Food and drink',df['Food and drink'].cast('string'))
df = df.withColumn('Online boarding',df['Online boarding'].cast('string'))
df = df.withColumn('Seat comfort',df['Seat comfort'].cast('string'))
df = df.withColumn('Inflight entertainment',
                                         df['Inflight entertainment'].cast('string'))
df = df.withColumn('On-board service',df['On-board service'].cast('string'))
df = df.withColumn('Leg room service',df['Leg room service'].cast('string'))
df = df.withColumn('Baggage handling',df['Baggage handling'].cast('string'))
df = df.withColumn('Checkin service',df['Checkin service'].cast('string'))                                                                          
df = df.withColumn('Inflight service',df['Inflight service'].cast('string'))                                                                          
df = df.withColumn('Cleanliness',df['Cleanliness'].cast('string'))    
df = df.withColumn('Departure/Arrival time convenient',
                                         df['Departure/Arrival time convenient'].cast('string'))  

In [ ]:
from pyspark.sql.functions import isnan,when,count,col
df = df.count()
df.select([count(when(isnan(c)| col(c).isNull(),c)).alias(c) for c in df.columns]).show()


In [ ]:
import pyspark.sql.functions as f
from pyspark.sql.types import *
def DepartureDelayRemark(value):
    if   value <= 45: 
        return '0'
    elif value > 45:
        return '1'

In [ ]:
depart= f.udf(DepartureDelayRemark,StringType())
df = df.withColumn('Departure_Delay', 
                                         depart('Departure Delay in Minutes'))


In [ ]:
def ArrivalDelayRemark(value):
    if   value <= 45: 
        return '0'
    elif value > 45:
        return '1'

In [ ]:
Arrival= f.udf(ArrivalDelayRemark,StringType())
df = df.withColumn('Arrival_Delay', Arrival('Arrival Delay in Minutes'))

In [ ]:
df = df.withColumn('satisfaction', 
                                       f.when(f.col('satisfaction') == 'satisfied', '1')
                                       .otherwise('0'))

In [ ]:
print(np.array(df.select('Gender').distinct().collect()).flatten())
print(np.array(df.select('Customer Type').distinct().collect()).flatten())
print(np.array(df.select('Type of Travel').distinct().collect()).flatten())
print(np.array(df.select('Class').distinct().collect()).flatten())

In [ ]:
df = df.withColumnRenamed("Customer Type","Customer_Type") \
    .withColumnRenamed("Type of Travel","Type_of_Travel")


In [ ]:
df = df.withColumn('Gender', 
                                       f.when(f.col('Gender') == 'Female', '1')
                                       .otherwise('0'))

In [ ]:
df = df.withColumn('Customer_Type', 
                                       f.when(f.col('Customer_Type') == 'Loyal Customer', '1')
                                       .otherwise('0'))

In [ ]:
df = df.withColumn('Type_of_Travel', 
                                       f.when(f.col('Type_of_Travel') == 'Personal Travel',
                                              '1').otherwise('0'))

In [ ]:
from pyspark.ml.feature import StringIndexer

In [ ]:
def Class_reclassify(x):
    if x == 'Eco':
        return '0'
    if x == 'Eco Plus':
        return '1'
    if x == 'Business':
        return '2'

In [ ]:
class_re = f.udf(Class_reclassify,StringType())
df = df.withColumn('Class',class_re('Class'))

In [ ]:
df.write.csv('textdata.csv')